# Agentic Evaluations


In [2]:
import os
import logging
from logging import warnings
from loguru import logger

logger.add("logs/vector_store.log")
logger.info("Starting Vector Store Notebook")

from dotenv import load_dotenv, find_dotenv
import nest_asyncio
nest_asyncio.apply()
load_dotenv(find_dotenv())

warnings.filterwarnings('ignore')
logging.getLogger().setLevel(logging.ERROR)

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import StorageContext
from IPython.display import Markdown, display

from llama_index.vector_stores.qdrant import QdrantVectorStore
import qdrant_client

warnings.filterwarnings('ignore')
logging.getLogger().setLevel(logging.ERROR)

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import StorageContext
from IPython.display import Markdown, display

from llama_index.vector_stores.qdrant import QdrantVectorStore
import qdrant_client

2025-03-29 17:33:00.339 | INFO     | __main__:<module>:7 - Starting Vector Store Notebook


In [3]:
QDRANT_HOSTED_URL = os.getenv("QDRANT_HOSTED_URL")
QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")
client = qdrant_client.QdrantClient(
    QDRANT_HOSTED_URL,
    api_key=QDRANT_API_KEY,
)

In [4]:
from pathlib import Path

notebook_dir = Path().absolute()
file_path = str(notebook_dir / ".." / "data" / "paper" / "stanford-cynicism.pdf")

documents = SimpleDirectoryReader(input_files=[file_path]).load_data()

In [5]:
len(documents), type(documents[0])

(11, llama_index.core.schema.Document)

In [6]:
documents[0].metadata

{'page_label': '1',
 'file_name': 'stanford-cynicism.pdf',
 'file_path': '/Users/pmui/SynologyDrive/research/2025/research2025/06-advanced-rag/notebooks/../data/paper/stanford-cynicism.pdf',
 'file_type': 'application/pdf',
 'file_size': 490320,
 'creation_date': '2025-03-22',
 'last_modified_date': '2025-03-22'}

In [7]:
from pprint import pprint
pprint(documents[0].text[:200])

(' Semantic Analysis of Diversity Rhetoric and \n'
 ' Ideological Contraction \n'
 ' in Stanford Daily \n'
 ' Theodore Mui \n'
 ' theodoremui@gmail.com \n'
 ' Abstract \n'
 ' Cynicism, a distrust of people and institutions, creates ')


In [8]:
[doc.text[:80].replace("\n", "") for doc in documents]

[' Semantic Analysis of Diversity Rhetoric and  Ideological Contraction  in Stan',
 ' This study focuses specifically on opinion pieces published in the Stanford Dai',
 ' text-embedding-3-large  model.  In this embedding space, we then tally up the p',
 ' MFT posits that moral judgments are often rooted in key psychological foundatio',
 ' per year, the regression line (first graph on the left below) for normalized di',
 ' while the linguistic markers of diversity proliferate, the actual range of pers',
 ' class  MFTAnalysis  (BaseModel):  authority_subversion: float = Field(  defau',
 ' Let’s look at observations in  Figure 3  .  The trendlines for all three indic',
 ' Discussion  Figure 3  shows that measures of cynicism are increasing  in Stanf',
 ' ●  If students feel like only certain viewpoints are accepted, they may believe',
 ' Haidt, J.  The Righteous Mind: Why Good People are Divided By Politics and Reli']

In [9]:
index = VectorStoreIndex.from_documents(documents, show_progress=True)

Generating embeddings: 100%|██████████| 11/11 [00:00<00:00, 12.52it/s]


In [11]:
query_engine = index.as_query_engine()
response = query_engine.query("What is the main idea of the paper?")
display(Markdown(response.response))

The main idea of the paper is to analyze the growth of cynicism within the opinions section of the Stanford Daily over a 15-year period, focusing on how the narrowing range of topics discussed reflects a divergence between the rhetoric of diversity and the actual space for ideas. The study aims to understand how this discrepancy may lead to the development of cynical attitudes towards diversity programs and institutions, emphasizing the importance of fostering a more open and trusting academic environment through inclusive conversations and diverse viewpoints.

## How do we know that our RAG is any good?


In [12]:
from llama_index.core.chat_engine.types import ChatMode
from llama_index.core.vector_stores.types import VectorStoreQueryMode
from llama_index.core.response_synthesizers import get_response_synthesizer, ResponseMode

better_query_engine = index.as_query_engine(
    similarity_top_k=8, 
    vector_store_query_mode=VectorStoreQueryMode.MMR, # Max Marginal Relevance
    vector_store_query_kwargs={"mmr_prefetch_k": 16},
    response_mode=ResponseMode.ACCUMULATE
)
response = query_engine.query("What is the main idea of the paper?")
display(Markdown(response.response))

The main idea of the paper is to analyze the growth of cynicism within the opinions section of the Stanford Daily over a 15-year period, focusing on how the narrowing range of topics discussed reflects a divergence between the rhetoric of diversity and the actual space for diverse ideas. The study aims to understand how this discrepancy may lead to cynical attitudes towards diversity programs and institutions, emphasizing the importance of fostering a more open and trusting academic environment through inclusive conversations and welcoming diverse opinions.